# 03 · Model Training
Train a small PyTorch MLP and evaluate calibration.

In [ ]:
import numpy as np
from oncotarget.io import merge_gene_feature_table
from oncotarget.features import create_feature_matrix
from oncotarget.model import train_mlp, MLPConfig
from oncotarget.eval import classification_summary, summarize_bootstrap, compute_reliability_curve

merged = merge_gene_feature_table()
feature_matrix = create_feature_matrix(merged)
metrics, model = train_mlp(feature_matrix.features, feature_matrix.labels, MLPConfig(epochs=100, patience=10))
metrics


In [ ]:
import torch
with torch.no_grad():
    logits = model(torch.tensor(feature_matrix.features.values, dtype=torch.float32))
    probs = torch.sigmoid(logits).numpy().squeeze()
y_true = feature_matrix.labels.astype(int).to_numpy()
classification_summary(y_true, probs)


In [ ]:
bootstrap = summarize_bootstrap(y_true, probs, n_bootstraps=64)
{k: v.__dict__ for k, v in bootstrap.items()}


In [ ]:
calibration_df = compute_reliability_curve(y_true, probs)
calibration_df
